# Carga de librerias

In [101]:
from elasticsearch import Elasticsearch
import time
from datetime import timedelta, datetime
from lorem_text import lorem

es = Elasticsearch("http://127.0.0.1:9200")

print(f"Connected to ElasticSearch cluster `{es.info().body['cluster_name']}`")

Connected to ElasticSearch cluster `docker-cluster`


# Actualizaciones a la base de datos con 1 millón de registros y todos los campos indexados

En esta sección se revisará la operación de actualización para la colección de 1 millón de documentos ya indexados

In [102]:
idx = "extracto_cuenta_1m"
es.indices.get_mapping(index=idx)

ObjectApiResponse({'extracto_cuenta_1m': {'mappings': {'properties': {'agencia': {'type': 'keyword'}, 'descripcion': {'type': 'text'}, 'fecha': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss'}, 'monto': {'type': 'float'}, 'nota': {'type': 'text'}, 'saldo': {'type': 'float'}}}}})

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [59]:
def actualizacion_fecha_monto(start_date, end_date, monto_actualizar):
      from_date_str = f"{start_date}" + " 00:00:00"
      to_date_str = f"{end_date}" + " 23:59:59"
   

      script = {
            "source": f"ctx._source['monto'] = {monto_actualizar}; ctx._source['nota'] = 'Actualización de monto por fecha'; ",
            "lang": "painless"
      }

      query = {
                  "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
                  }
      }

      return es.options(request_timeout=60).update_by_query(query=query, script=script, index=idx)

In [13]:
start_date = "2021-07-01"
end_date = "2021-07-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1175 documentos
 Actualización realizada en un tiempo de 0:00:04.092742


In [16]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
    "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
      }
  }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Retención IVA', 'fecha': '2021-07-14 05:38:36', 'monto': 7890, 'saldo': 4376768.06, 'nota': 'Actualización de monto por fecha', 'agencia': 'Santa Cruz'}
{'descripcion': 'Ingreso en efectivo', 'fecha': '2021-07-10 21:26:13', 'monto': 7890, 'saldo': 4989208.43, 'nota': 'Actualización de monto por fecha', 'agencia': 'Beni'}
{'descripcion': 'Pago en interés', 'fecha': '2021-07-07 03:26:50', 'monto': 7890, 'saldo': 4070623.65, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Apertura de cuenta corriente', 'fecha': '2021-07-30 00:22:00', 'monto': 7890, 'saldo': 2384287.36, 'nota': 'Actualización de monto por fecha', 'agencia': 'Tarija'}
{'descripcion': 'Capitalización de intereses premio', 'fecha': '2021-07-25 14:02:02', 'monto': 7890, 'saldo': 9181104.58, 'nota': 'Actualización de monto por fecha', 'agencia': 'Santa Cruz'}
{'descripcion': 'Compra en comercio', 'fecha': '2021-07-06 03:15:50', 'monto': 7890, 'saldo': 4954872.26, 'nota': 'A

## Actualización 2: Nota por descripción y fecha

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [49]:
def actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto_actualizar):
    from_date_str = start_date + " 00:00:00"
    to_date_str = end_date + " 23:59:59"
    
    script = {
            "source": f"ctx._source['monto'] = {monto_actualizar}; ctx._source['nota'] = 'Actualización de monto por descripción'; ",
            "lang": "painless"
      }


    query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }
    
    return es.options(request_timeout=30).update_by_query(query=query, script=script, index=idx)

In [104]:
start_date = "1990-05-14"
end_date = "1990-05-25"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 234 documentos
 Actualización realizada en un tiempo de 0:00:05.068827


In [105]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05-15 10:54:20', 'monto': 10.5, 'saldo': 9195278.66, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05-23 15:24:51', 'monto': 10.5, 'saldo': 7979192.79, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05-21 19:54:52', 'monto': 10.5, 'saldo': 8282296.57, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05-23 04:08:15', 'monto': 10.5, 'saldo': 3648083.1, 'nota': 'Actualización de monto por descripción', 'agencia': 'Pando'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05-18 17:11:53', 'monto': 10.5, 'saldo': 4276613.13, 'nota': 'Actualización de monto por descripción', 'agencia': 'La Paz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1990-05

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [56]:
def actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion):
    query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

    script = {
            "source": f"ctx._source['saldo'] = {saldo}; ctx._source['nota'] = 'Actualización de saldo por agencia y monto'; ",
            "lang": "painless"
      }

    return es.options(request_timeout=30).update_by_query(query=query, script=script, index=idx)

In [30]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 50 documentos
 Actualización realizada en un tiempo de 0:00:00.731463


In [31]:
query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Intereses', 'fecha': '2014-04-21 10:07:10', 'monto': 1062.11, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1968-04-13 11:40:35', 'monto': 579.58, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1979-03-29 21:48:29', 'monto': 1486.64, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1970-04-11 01:54:56', 'monto': 1130.29, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1974-02-13 07:04:30', 'monto': 1148.86, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1952-08-23 01:11:59', 'monto': 648.56, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'a

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 4.0927 s       |
| Tiempo actualización 2      | 2.0756 s       |
| Tiempo actualización 3      | 0.7315 s       |

---------------------

# Actualizaciones a la base de datos con 10 millones de registros y todos los campos indexados

En esta sección se revisará la operación de actualización para la colección de 10 millones de documentos indexados.

In [32]:
idx = "extracto_cuenta_10m"
es.indices.get_mapping(index=idx)

ObjectApiResponse({'extracto_cuenta_10m': {'mappings': {'properties': {'agencia': {'type': 'keyword'}, 'descripcion': {'type': 'text'}, 'fecha': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss'}, 'monto': {'type': 'float'}, 'nota': {'type': 'text'}, 'saldo': {'type': 'float'}}}}})

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [60]:
start_date = "2010-08-01"
end_date = "2010-08-31"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 11768 documentos
 Actualización realizada en un tiempo de 0:00:05.333148


In [61]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
    "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
      }
  }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Retiro en efectivo', 'fecha': '2010-08-01 03:10:35', 'monto': 7890, 'saldo': 6338413.4, 'nota': 'Actualización de monto por fecha', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de cuenta corriente', 'fecha': '2010-08-13 22:26:29', 'monto': 7890, 'saldo': 3657263.72, 'nota': 'Actualización de monto por fecha', 'agencia': 'Beni'}
{'descripcion': 'Transferencia interbancaria SIMPLE', 'fecha': '2010-08-28 02:46:27', 'monto': 7890, 'saldo': 6609921.69, 'nota': 'Actualización de monto por fecha', 'agencia': 'Pando'}
{'descripcion': 'Aporte inversor', 'fecha': '2010-08-12 11:18:20', 'monto': 7890, 'saldo': 4836288.31, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Retención IVA', 'fecha': '2010-08-24 10:42:34', 'monto': 7890, 'saldo': 9673117.63, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'AB.CTA.P/MOVIL', 'fecha': '2010-08-04 19:18:48', 'monto': 7890, 'saldo': 4241415.5, 'nota': 'Actualización

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [53]:
start_date = "1981-01-14"
end_date = "1981-01-20"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1335 documentos
 Actualización realizada en un tiempo de 0:00:13.778107


In [54]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-16 15:54:10', 'monto': 10.5, 'saldo': 4970158.64, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-16 19:23:37', 'monto': 10.5, 'saldo': 9581197.33, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-18 06:43:58', 'monto': 10.5, 'saldo': 5780232.94, 'nota': 'Actualización de monto por descripción', 'agencia': 'Beni'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-15 18:21:33', 'monto': 10.5, 'saldo': 1203279.79, 'nota': 'Actualización de monto por descripción', 'agencia': 'Pando'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-15 21:08:07', 'monto': 10.5, 'saldo': 4815576.98, 'nota': 'Actualización de monto por descripción', 'agencia': 'Oruro'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-19 10

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [57]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 496 documentos
 Actualización realizada en un tiempo de 0:00:09.869640


In [58]:
query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Intereses', 'fecha': '2014-04-21 10:07:10', 'monto': 1062.11, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1968-04-13 11:40:35', 'monto': 579.58, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1979-03-29 21:48:29', 'monto': 1486.64, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1970-04-11 01:54:56', 'monto': 1130.29, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1974-02-13 07:04:30', 'monto': 1148.86, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1952-08-23 01:11:59', 'monto': 648.56, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'a

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 5.3331 s       |
| Tiempo actualización 2      | 13.7781 s      |
| Tiempo actualización 3      | 9.8696 s       |

------------------------------------

# Actualizaciones a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de actualización para la colección de 30 millones de documentos indexados.

In [73]:
idx = "extracto_cuenta_30m"
es.indices.get_mapping(index=idx)

ObjectApiResponse({'extracto_cuenta_30m': {'mappings': {'properties': {'agencia': {'type': 'keyword'}, 'descripcion': {'type': 'text'}, 'fecha': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss'}, 'monto': {'type': 'float'}, 'nota': {'type': 'text'}, 'saldo': {'type': 'float'}}}}})

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [77]:
start_date = "2009-12-01"
end_date = "2009-12-01"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1169 documentos
 Actualización realizada en un tiempo de 0:00:48.078708


In [78]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
    "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
      }
  }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Capitalización de intereses premio', 'fecha': '2009-12-01 03:21:57', 'monto': 7890, 'saldo': 4906592.89, 'nota': 'Actualización de monto por fecha', 'agencia': 'Oruro'}
{'descripcion': 'Pago de cheque compensado', 'fecha': '2009-12-01 03:41:19', 'monto': 7890, 'saldo': 2817630.43, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Retención IVA', 'fecha': '2009-12-01 21:35:30', 'monto': 7890, 'saldo': 9094789.57, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Reintegro cajero automático', 'fecha': '2009-12-01 04:54:00', 'monto': 7890, 'saldo': 4120074.9, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Debito Cta por pago de Seguro', 'fecha': '2009-12-01 21:22:43', 'monto': 7890, 'saldo': 3125225.62, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Intereses', 'fecha': '2009-12-01 22:04:08', 'monto': 7890, 'saldo': 6444708.

## Actualización 2: Nota por descripción

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [79]:
start_date = "1981-01-14"
end_date = "1981-01-14"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 545 documentos
 Actualización realizada en un tiempo de 0:00:05.441910


In [80]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 14:31:27', 'monto': 10.5, 'saldo': 5574509.46, 'nota': 'Actualización de monto por descripción', 'agencia': 'Beni'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 11:37:03', 'monto': 10.5, 'saldo': 2958007.97, 'nota': 'Actualización de monto por descripción', 'agencia': 'Cochabamba'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 14:37:53', 'monto': 10.5, 'saldo': 6136543.39, 'nota': 'Actualización de monto por descripción', 'agencia': 'Beni'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 18:54:04', 'monto': 10.5, 'saldo': 8080093.49, 'nota': 'Actualización de monto por descripción', 'agencia': 'Pando'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 16:02:24', 'monto': 10.5, 'saldo': 8956844.5, 'nota': 'Actualización de monto por descripción', 'agencia': 'Chuquisaca'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [81]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1275 documentos
 Actualización realizada en un tiempo de 0:00:03.250859


In [82]:
query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Intereses', 'fecha': '1961-08-18 10:01:42', 'monto': 1032.94, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1981-08-30 12:57:39', 'monto': 941.13, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1983-10-14 00:31:38', 'monto': 810.47, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1967-01-21 19:24:26', 'monto': 707.38, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1990-05-08 23:20:24', 'monto': 1953.31, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1985-09-01 15:50:23', 'monto': 204.74, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'age

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 48.0787 s      |
| Tiempo actualización 2      | 5.4419 s       |
| Tiempo actualización 3      | 3.2508 s       |

--------------------

# Actualizaciones a la base de datos con 10 millones de registros y 3 campos indexados (fecha, descripcion, nota)

En esta sección se revisará la operación de actualización para la colección de 10 millones de documentos indexados en tres campos: fecha, descripcion, nota.

In [83]:
idx = "extracto_cuenta_10m_3idx"
es.indices.get_mapping(index=idx)

ObjectApiResponse({'extracto_cuenta_10m_3idx': {'mappings': {'properties': {'agencia': {'type': 'keyword', 'index': False}, 'descripcion': {'type': 'text'}, 'fecha': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss'}, 'monto': {'type': 'float', 'index': False}, 'nota': {'type': 'text'}, 'saldo': {'type': 'float', 'index': False}}}}})

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [86]:
start_date = "2009-12-01"
end_date = "2009-12-05"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 2021 documentos
 Actualización realizada en un tiempo de 0:00:28.189595


In [87]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
    "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
      }
  }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Capitalización de intereses premio', 'fecha': '2009-12-03 01:22:10', 'monto': 7890, 'saldo': 5782096.86, 'nota': 'Actualización de monto por fecha', 'agencia': 'Cochabamba'}
{'descripcion': 'Capitalización de intereses premio', 'fecha': '2009-12-04 18:40:23', 'monto': 7890, 'saldo': 5512900.73, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Retiro en efectivo', 'fecha': '2009-12-03 15:09:23', 'monto': 7890, 'saldo': 5977062.44, 'nota': 'Actualización de monto por fecha', 'agencia': 'Chuquisaca'}
{'descripcion': 'Ingreso en efectivo', 'fecha': '2009-12-03 11:33:15', 'monto': 7890, 'saldo': 783330.78, 'nota': 'Actualización de monto por fecha', 'agencia': 'Oruro'}
{'descripcion': 'Capitalización de intereses premio', 'fecha': '2009-12-01 03:21:57', 'monto': 7890, 'saldo': 4906592.89, 'nota': 'Actualización de monto por fecha', 'agencia': 'Oruro'}
{'descripcion': 'Deposito a cuenta', 'fecha': '2009-12-04 12:22:41', 'monto': 7890, 'sa

In [88]:
start_date = "1981-01-14"
end_date = "1981-01-14"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 201 documentos
 Actualización realizada en un tiempo de 0:00:02.707747


In [89]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 02:31:45', 'monto': 10.5, 'saldo': 5857589.96, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 20:58:41', 'monto': 10.5, 'saldo': 232370.51, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 05:03:32', 'monto': 10.5, 'saldo': 2830562.61, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 19:14:06', 'monto': 10.5, 'saldo': 847023.48, 'nota': 'Actualización de monto por descripción', 'agencia': 'La Paz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 13:13:01', 'monto': 10.5, 'saldo': 3098863.51, 'nota': 'Actualización de monto por descripción', 'agencia': 'Pando'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 0

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [90]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 2000

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 453 documentos
 Actualización realizada en un tiempo de 0:00:05.372324


In [91]:
query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Intereses', 'fecha': '2014-04-21 10:07:10', 'monto': 1062.11, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1968-04-13 11:40:35', 'monto': 579.58, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1979-03-29 21:48:29', 'monto': 1486.64, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1970-04-11 01:54:56', 'monto': 1130.29, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1974-02-13 07:04:30', 'monto': 1148.86, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1952-08-23 01:11:59', 'monto': 648.56, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'a

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 28.1896 s      |
| Tiempo actualización 2      | 2.7077 s       |
| Tiempo actualización 3      | 5.3723 s       |

----------------------------------

# Actualizaciones a la base de datos con 30 millones de registros y todos los campos indexados

En esta sección se revisará la operación de actualización para la colección de 30 millones de documentos indexados.

In [92]:
idx = "extracto_cuenta_30m_3idx"
es.indices.get_mapping(index=idx)

ObjectApiResponse({'extracto_cuenta_30m_3idx': {'mappings': {'properties': {'agencia': {'type': 'keyword', 'index': False}, 'descripcion': {'type': 'text'}, 'fecha': {'type': 'date', 'format': 'yyyy-MM-dd HH:mm:ss'}, 'monto': {'type': 'float', 'index': False}, 'nota': {'type': 'text'}, 'saldo': {'type': 'float', 'index': False}}}}})

## Actualización 1: Monto por fecha

Se actualizarán los campos monto y nota, dependiendo de una fecha especificada

In [94]:
start_date = "2008-12-01"
end_date = "2008-12-01"
monto_actualizar = 7890

start_time = time.time()
results = actualizacion_fecha_monto(start_date, end_date, monto_actualizar)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 1118 documentos
 Actualización realizada en un tiempo de 0:00:34.923516


In [95]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
    "range" : {
                  "fecha": { "gte" : f"{from_date_str}", "lte" : f"{to_date_str}" },
      }
  }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Pago de servicio telefónico', 'fecha': '2008-12-01 01:03:01', 'monto': 7890, 'saldo': 5406024.36, 'nota': 'Actualización de monto por fecha', 'agencia': 'Beni'}
{'descripcion': 'Compra en comercio', 'fecha': '2008-12-01 22:31:57', 'monto': 7890, 'saldo': 8919887.11, 'nota': 'Actualización de monto por fecha', 'agencia': 'Cochabamba'}
{'descripcion': 'Retención IVA', 'fecha': '2008-12-01 23:43:08', 'monto': 7890, 'saldo': 9263142.44, 'nota': 'Actualización de monto por fecha', 'agencia': 'La Paz'}
{'descripcion': 'AB.CTA.P/MOVIL', 'fecha': '2008-12-01 19:13:28', 'monto': 7890, 'saldo': 1458272.52, 'nota': 'Actualización de monto por fecha', 'agencia': 'Pando'}
{'descripcion': 'Pago en línea - Servicio de internet', 'fecha': '2008-12-01 21:51:43', 'monto': 7890, 'saldo': 9359827.69, 'nota': 'Actualización de monto por fecha', 'agencia': 'La Paz'}
{'descripcion': 'Pago recibo de luz', 'fecha': '2008-12-01 21:47:58', 'monto': 7890, 'saldo': 4318437.33, 'nota': 'Actualizaci

## Actualización 2: Nota por descripción
Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [96]:
start_date = "1981-01-14"
end_date = "1981-01-14"
descripcion1 = 'Aporte inversor'
descripcion2 = 'Apertura de caja de ahorros'
monto = 10.50

start_time = time.time()
results = actualizacion_monto_descripcion_fecha(start_date, end_date, descripcion1, descripcion2, monto)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 545 documentos
 Actualización realizada en un tiempo de 0:00:29.347907


In [97]:
from_date_str = f"{start_date}" + " 00:00:00"
to_date_str = f"{end_date}" + " 23:59:59"

query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"descripcion": descripcion1}},
                            {"match": {"descripcion": descripcion2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "fecha": {
                                "gte": from_date_str,
                                "lte" : to_date_str 
                        },
                    }
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 02:31:45', 'monto': 10.5, 'saldo': 5857589.96, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 20:58:41', 'monto': 10.5, 'saldo': 232370.51, 'nota': 'Actualización de monto por descripción', 'agencia': 'Tarija'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 05:03:32', 'monto': 10.5, 'saldo': 2830562.61, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 16:22:45', 'monto': 10.5, 'saldo': 9903779.97, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '1981-01-14 14:22:02', 'monto': 10.5, 'saldo': 4603146.48, 'nota': 'Actualización de monto por descripción', 'agencia': 'Santa Cruz'}
{'descripcion': 'Apertura de caja de ahorros', 'fecha': '19

## Actualización 3: Saldo por agencias, monto y descripcion

Se actualizarán los campos monto y nota, dependiendo de una descripción especificada.

In [99]:
agencia1 = "Oruro"
agencia2 = "Potosi"
descripcion = 'Intereses'
saldo = 1234.5
monto = 200

start_time = time.time()
results = actualizacion_saldo_monto_agencia(saldo, monto, agencia1, agencia2, descripcion)
end_time = time.time()

print("="*100)
print(f" Se han actualizado {results['total']} documentos")
print("="*100)
print(f" Actualización realizada en un tiempo de {str(timedelta(seconds = (end_time - start_time)))}")
print("="*100)

 Se han actualizado 119 documentos
 Actualización realizada en un tiempo de 0:00:01.960684


In [100]:
query = {
        "bool": {
            "must": [
                {
                    "bool": {
                        "should": [
                            {"match": {"agencia": agencia1}},
                            {"match": {"agencia": agencia2}}
                        ]
                    }
                },
                {
                    "range" : {
                        "monto": {
                                "lt" : monto 
                        },
                    }
                },
                {
                    "match": {"descripcion": descripcion}
                }
            ]
        }
    }

results = es.search(index=idx, query=query)

for hit in results['hits']['hits']:
    print(hit["_source"])

print(f"Se encontraron: {results['hits']['total']['value']} resultados")

{'descripcion': 'Intereses', 'fecha': '2017-11-01 13:08:13', 'monto': 126.87, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1955-09-03 15:58:54', 'monto': 172.05, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1969-04-27 15:44:04', 'monto': 138.31, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1954-09-28 14:45:39', 'monto': 70.05, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '1984-08-01 05:16:38', 'monto': 40.56, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia': 'Oruro'}
{'descripcion': 'Intereses', 'fecha': '2014-10-10 17:11:01', 'monto': 167.53, 'saldo': 1234.5, 'nota': 'Actualización de saldo por agencia y monto', 'agencia

| Tabla resumen de resultados | Valor obtenido |
|-----------------------------|----------------|
| Tiempo actualización 1      | 34.9235 s      |
| Tiempo actualización 2      | 29.3479 s      |
| Tiempo actualización 3      | 1.9607 s       |